In [ ]:
import html
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from wordcloud import WordCloud
import plotly.express as px
import nltk
from nltk import word_tokenize, pos_tag

nltk.download('punkt')

In [ ]:
df = pd.read_csv('../datasets/emscad_v1.csv')

In [ ]:
df.columns

In [ ]:
df[["telecommuting", "has_company_logo","has_questions", "fraudulent", "in_balanced_dataset"]].head(5)

### Distribution of Null values for fraudulent against all dataset

In [ ]:
# count all the columns that have null values
df.isnull().sum()[df.isnull().sum()>0].sort_values(ascending=False)

In [ ]:
# count all the columns that have null values that are fraudulent
df[df["fraudulent"]=="t"].isnull().sum()[df[df["fraudulent"]=="t"].isnull().sum()>0].sort_values(ascending=False)

In [ ]:
# percentage difference of null values between fraudulent and total
fraud = df[df["fraudulent"]=="t"].isnull().sum()[df[df["fraudulent"]=="t"].isnull().sum()>0]
total = df.isnull().sum()[df.isnull().sum()>0]
f = (fraud/total*100).sort_values(ascending=False)
f

In [ ]:
# plot a bar chart of the percentage difference of null values between fraudulent and total
# label each bar with the percentage
plt.figure(figsize=(10,5))
[plt.text(v, i, str(round(v,2))+"%") for i, v in enumerate(f)]
plt.barh(f.index, f)
plt.title("Percentage of null values in fraudulent out of total job postings")
plt.xlabel("Percentage")
plt.ylabel("Columns")
plt.show()


### Word Cloud for fraudulent and non-fraudulent transactions

In [ ]:
fraud_df_desc = df[df["fraudulent"]=="t"]["description"]
fraud_df_desc

In [ ]:
def remove_html_tags_and_escape_chars(input_text):
    # Remove HTML tags
    text_without_html = BeautifulSoup(input_text, 'html.parser').get_text()

    # Unescape HTML characters
    text_without_escape_chars = html.unescape(text_without_html)

    return text_without_escape_chars

fraud_df_desc = fraud_df_desc.apply(remove_html_tags_and_escape_chars)

In [ ]:
fraud_df_desc

In [ ]:
def remove_non_alpha(input_text):
    return ''.join(char if char.isalpha() or char.isspace() else ' ' for char in input_text)
def remove_newlines(text):
    return text.replace('\n', '').replace('\r', '')
fraud_df_desc = fraud_df_desc.apply(remove_non_alpha)
fraud_df_desc = fraud_df_desc.apply(remove_newlines)


In [ ]:
fraud_df_desc = fraud_df_desc.str.lower()

In [ ]:
fraud_df_desc =fraud_df_desc.apply(lambda x: word_tokenize(x.lower()))
fraud_df_desc = fraud_df_desc.apply(lambda x: [word for word in x if len(word) >= 3])
fraud_df_desc

In [ ]:
fraud_df_desc

In [ ]:
# put all the words into a string
fraud_words = ""
for i in fraud_df_desc:
    fraud_words += " ".join(i) + " "


In [ ]:
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(fraud_words)

In [ ]:
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')  # Turn off axis labels
plt.show()

Uses Pos Tagging to further filter out words

In [ ]:
# Perform part-of-speech tagging
def only_adj(x):
    pos_tags = pos_tag(x)
    return [word for word, pos in pos_tags if pos in ['JJ', 'JJR', 'JJS']]

In [ ]:
df_adj = fraud_df_desc.apply(lambda x: only_adj(x))
df_adj = df_adj[df_adj.apply(lambda x: len(x) > 0)]

In [ ]:
# put all the words into a string
fraud_words = ""
for i in df_adj:
    fraud_words += " ".join(i) + " "

In [ ]:
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(fraud_words)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')  # Turn off axis labels
plt.show()

### Number of fraudulent job postings per state

In [ ]:
df_loc = df[df["location"].notnull()][["location", "fraudulent"]]

In [ ]:
df_loc['location'] = df_loc['location'].str.split(', ')
df_loc['location'] = df_loc['location'].apply(lambda x: x if len(x)==3 else np.nan)
df_loc = df_loc[df_loc['location'].notnull()]

In [ ]:
df_loc[['Country', 'State', 'City']] = pd.DataFrame(df_loc['location'].tolist(), index=df_loc.index)

In [ ]:
df_loc = df_loc[["Country","State", "City", "fraudulent"]]

In [ ]:
# drop all rows with na
df_loc.replace('', np.nan, inplace=True)
df_loc = df_loc.dropna()

In [ ]:
df_loc["fraudulent"] = df_loc["fraudulent"].apply(lambda x: 1 if x=="t" else 0)

In [ ]:
# find the percentage of fraudulent job postings out of all in each state
fraudulent_counts = df_loc.groupby("State")["fraudulent"].sum().reset_index()

In [ ]:
# plot a map using python's plotly library
# also show the number of fraudulent job postings per state
fig = px.choropleth(fraudulent_counts, locations="State",
                    locationmode="USA-states",
                    color="fraudulent",
                    scope="usa",
                    title="Number of fraudulent job postings per state",
                    color_continuous_scale=px.colors.sequential.OrRd,
                    labels={'fraudulent':'Number of fraudulent job postings'},
                    template="plotly_white",
                    width=1000,
                    height=600)
# Add scatter points with text values
fig.add_trace(px.scatter_geo(fraudulent_counts,
                            locations='State',
                            text='fraudulent',
                            locationmode='USA-states',
                            color="fraudulent",
                            ).data[0])

fig.show()
